### 패키지 불러오기

In [ ]:
import pandas as pd
import pyautogui as pg
import win32com.client
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import clipboard
import math

### EP 로그인

In [ ]:
my_id = pg.prompt(title="id")
my_pw = pg.password(title="EP_PASS", text="패스워드를 입력하세요", default="")

url = "http://swpsso.posco.net/idms/U61/jsp/login/login.jsp"

driver = webdriver.Edge("msedgedriver")


driver.get(url)
driver.maximize_window()

driver.find_element(By.XPATH,'//*[@id="username"]').send_keys(my_id)
driver.find_element(By.XPATH, '//*[@id="password"]').send_keys(my_pw)
driver.find_element(By.XPATH, '//*[@id="loginsubmit"]').click()
driver.implicitly_wait(10)

pg.sleep(5)
try:
    alert=driver.switch_to.alert # alert(경고창) 허용
    alert.accept()
except:
    print("기존 로그인 경고창 없음.")

pg.sleep(2) #슬립이 없으면 에러
try:
    for i in reversed(range(len(driver.window_handles))): # 현재 활성화된 웹드라이버 핸들 중 제목이 EP가 아닌 드라이버는 종료
        driver.switch_to.window(driver.window_handles[i])
        if driver.title != 'EP(Enterprise Portal)':
            driver.close()
except:
    print("팝업창 없음-계속 진행")

### 엑셀 데이터 활용 
- 엑셀파일 열기

In [ ]:
pg.hotkey('win','r') # 윈도우 실행
pg.write('D:\RPA\PIUM_list(test).xls') # 엑셀 경로 및 파일명 입력
pg.sleep(2)
pg.press('enter')
pg.sleep(5)

win=pg.getWindowsWithTitle('PIUM_list(test)')[0] # 해당 파일의 이름을 가진 창을 확대
if win.isMaximized == False:
    win.maximize()


- 엑셀파일 복사해서 파이썬에서 판다스로 읽기

In [ ]:
win.activate()

pg.hotkey('ctrl','home') # 워크시트 처음으로 이동
pg.hotkey('ctrl','a') # 전체 선택
pg.hotkey('ctrl','c') # 복사

list = pd.read_clipboard() # 클립보드에 복사한 내용을 파이썬 pandas로 읽기

list


- 참석방법 변수 지정

In [ ]:
lp = '러닝플랫폼'
con = '사운영회의'

### 러닝플랫폼 Sorting하여 메일 보내기

- 참석방법 : 러닝플랫폼으로 Sorting

In [ ]:
list_lp = list.loc[(list['참석방법'] == lp)] # 참석방법 Column에서 lp(러닝플랫폼)인 자료만 선택하여 list_lp변수에 저장

list_lp.loc[:, '이메일'] # list_lp자료 중 이메일 column만 sorting

email_lp=[] # 빈 리스트 생성
for i in list_lp.loc[:, '이메일']: # for문으로 이메일 column에 있는 이메일 하나씩 꺼내기
    email_lp.append(i)

list_lp.loc[:, '이메일']

eqa1 = math.ceil(len(list_lp)/200) # list_lp의 이메일 총 갯수에서 200개씩 나누어서 메일 보내야 할 횟수 지정

- 메일 아이콘 클릭하여 메일화면으로 접속

In [ ]:
driver.switch_to.window(driver.window_handles[-1]) # EP 창으로 전환

driver.find_element(By.XPATH,'//*[@id="533982"]/a[1]/span[1]/img').click() # 메일 아이콘 클릭
pg.sleep(3)

driver.switch_to.window(driver.window_handles[-1]) # 메일 창으로 전환

- 메일 본문 작성

In [ ]:
mail_num1 = 0

for i in range(eqa1):
    email_1=email_lp[mail_num1:mail_num1+200] # 슬라이싱(0~199번, 총 200개씩)
    email_1r=str(email_1).replace('[','').replace(']','') # 리스트 형태에서 [] 제거
    
    driver.switch_to.window(driver.window_handles[-1]) # 메일 창으로 전환 ------ 이게 빠져있어서 에러가 났나봅니다 ㅠㅠ
    driver.find_element(By.XPATH,'//*[@id="Lnb"]/div[1]/a').click() # 메일쓰기 아이콘 클릭

    pg.sleep(2)

    driver.switch_to.window(driver.window_handles[-1]) #메일쓰기 창으로 전환
    driver.find_element(By.XPATH,'//*[@id="token-input-send_to"]').send_keys(email_1r) # 받는 사람에 email_1r변수에 저장된 이메일 목록 붙이기
    
    pg.sleep(0.5)
    win=pg.getWindowsWithTitle('메일쓰기')[0] # 메일 쓰기 창을 가져와서 활성화
    win.activate()
    
    pg.press('enter')
    pg.sleep(1)

    title_lp = '[안내] PIUM과정 참석 안내(12.8/목 07:30~09:00) *참석방법: 러닝플랫폼' # 메일 제목 내용을 변수로 지정
    title_con = '[안내] PIUM과정 참석 안내(12.8/목 07:30~09:00) *참석방법: 사운영회의'

    driver.find_element(By.XPATH,'//*[@id="write_send_info"]/table[2]/tbody/tr[1]/td/input').send_keys(title_lp) # 메일 제목 붙이기
    pg.sleep(2)
    pg.press('tab', presses = 3)

    pg.hotkey('win','r') # 윈도우 실행버튼
    pg.write('D:\RPA\PIUM_mail.pptx') # '폴더경로\파일명' 입력
    pg.sleep(2)
    pg.press('enter')
    pg.sleep(5)

    win=pg.getWindowsWithTitle('PIUM_mail')[0] # ()안의 파일명과 동일한 창을 가져와서 창 최대화
    if win.isMaximized == False:
        win.maximize()

    pg.hotkey('ctrl','a') # 전체 선택
    pg.hotkey('ctrl','c') # 복사
    pg.sleep(2)

    win=pg.getWindowsWithTitle('메일쓰기')[0] # 메일 쓰기 창을 가져와서 활성화
    win.activate()

    pg.sleep(2)
    pg.press('tab') # 메일 제목에 있던 커서가 tab키 두번 눌러서 본문으로 이동
    pg.sleep(2)
    pg.hotkey('ctrl','v') # 위에서 복사해온 내용 붙여넣기

    pg.sleep(2)

    pg.press('tab',presses=11) # html 파일 붙여넣기
    pg.sleep(2)
    pg.press('enter')

    win=pg.getWindowsWithTitle('PIUM_mail')[0]
    win.activate()

    pg.press('PgDn') # 파워포인트 2번째 슬라이드로 이동
    pg.hotkey('ctrl','a')
    pg.hotkey('ctrl','c')
    pg.sleep(2)

    win_m=pg.getWindowsWithTitle('메일쓰기')[0] # 메일쓰기 창으로 이동
    win_m.activate()

    pg.sleep(2)
    pg.press('down') # 파워포인트 3번째 슬라이드 이동
    pg.hotkey('ctrl','v')

    pg.sleep(2)

    pg.press('tab',presses=5) # 이미지 붙여넣기
    pg.sleep(2)
    pg.write('700') # 이미지 사이즈 조정
    pg.sleep(2)
    pg.press('tab',presses=5) # 이미지 붙여넣기 버튼 클릭            
    pg.press('enter')
    pg.sleep(1)

    win.close()
    pg.sleep(1)

    driver.find_element(By.XPATH,'//*[@id="memo_content"]/div[1]/ul[1]/li[1]/a').click()

    pg.sleep(3)
    try:
        alert=driver.switch_to.alert
        alert.accept()
    except:
        print("발송확인 창 없음")
    
    pg.sleep(3)


    mail_num1+=200


### 사운영회의 Sorting하여 메일 보내기

- 참석방법 : 사운영회의로 Sorting

In [ ]:
list_con = list.loc[(list['참석방법'] == con)]

list_con.loc[:, '이메일']

email_con=[]
for i in list_con.loc[:, '이메일']:
    email_con.append(i)

list_con.loc[:, '이메일']

eqa2 = math.ceil(len(list_con)/200)

- 메일 아이콘 클릭하여 메일화면으로 접속

In [ ]:
driver.switch_to.window(driver.window_handles[-1]) # EP 창으로 전환

driver.find_element(By.XPATH,'//*[@id="533982"]/a[1]/span[1]/img').click() # 메일 아이콘 클릭
pg.sleep(3)

driver.switch_to.window(driver.window_handles[-1]) # 메일 창으로 전환

- 메일 본문 작성

In [ ]:
mail_num2 = 0

for i in range(eqa2):
    email_2=email_lp[mail_num2:mail_num2+200]
    email_2r=str(email_2).replace('[','').replace(']','')
    
    driver.switch_to.window(driver.window_handles[-1]) # 메일 창으로 전환 ------ 이게 빠져있어서 에러가 났나봅니다 ㅠㅠ
    driver.find_element(By.XPATH,'//*[@id="Lnb"]/div[1]/a').click() # 메일쓰기 아이콘 클릭

    pg.sleep(2)

    driver.switch_to.window(driver.window_handles[-1]) #메일쓰기 창으로 전환
    driver.find_element(By.XPATH,'//*[@id="token-input-send_to"]').send_keys(email_2r) # 받는 사람에 email_1r변수에 저장된 이메일 목록 붙이기
    
    pg.sleep(0.5)
    win=pg.getWindowsWithTitle('메일쓰기')[0] # 메일 쓰기 창을 가져와서 활성화
    win.activate()
    
    pg.press('enter')
    pg.sleep(1)

    title_lp = '[안내] PIUM과정 참석 안내(12.8/목 07:30~09:00) *참석방법: 러닝플랫폼' # 메일 제목 내용을 변수로 지정
    title_con = '[안내] PIUM과정 참석 안내(12.8/목 07:30~09:00) *참석방법: 사운영회의'

    driver.find_element(By.XPATH,'//*[@id="write_send_info"]/table[2]/tbody/tr[1]/td/input').send_keys(title_lp) # 메일 제목 붙이기
    pg.sleep(2)
    pg.press('tab', presses = 3)

    pg.hotkey('win','r') # 윈도우 실행버튼
    pg.write('D:\RPA\PIUM_mail.pptx') # '폴더경로\파일명' 입력
    pg.sleep(2)
    pg.press('enter')
    pg.sleep(5)

    win=pg.getWindowsWithTitle('PIUM_mail')[0] # ()안의 파일명과 동일한 창을 가져와서 창 최대화
    if win.isMaximized == False:
        win.maximize()

    pg.hotkey('ctrl','a') # 전체 선택
    pg.hotkey('ctrl','c') # 복사
    pg.sleep(2)

    win=pg.getWindowsWithTitle('메일쓰기')[0] # 메일 쓰기 창을 가져와서 활성화
    win.activate()

    pg.sleep(2)
    pg.press('tab') # 메일 제목에 있던 커서가 tab키 두번 눌러서 본문으로 이동
    pg.sleep(2)
    pg.hotkey('ctrl','v') # 위에서 복사해온 내용 붙여넣기

    pg.sleep(2)

    pg.press('tab',presses=11) # html 파일 붙여넣기
    pg.sleep(2)
    pg.press('enter')

    win=pg.getWindowsWithTitle('PIUM_mail')[0]
    win.activate()

    pg.press('PgDn') # 파워포인트 2번째 슬라이드로 이동
    pg.hotkey('ctrl','a')
    pg.hotkey('ctrl','c')
    pg.sleep(2)

    win_m=pg.getWindowsWithTitle('메일쓰기')[0] # 메일쓰기 창으로 이동
    win_m.activate()

    pg.sleep(2)
    pg.press('down') # 파워포인트 3번째 슬라이드 이동
    pg.hotkey('ctrl','v')

    pg.sleep(2)

    pg.press('tab',presses=5) # 이미지 붙여넣기
    pg.sleep(2)
    pg.write('700') # 이미지 사이즈 조정
    pg.sleep(2)
    pg.press('tab',presses=5) # 이미지 붙여넣기 버튼 클릭            
    pg.press('enter')
    pg.sleep(1)

    win.close()
    pg.sleep(1)

    driver.find_element(By.XPATH,'//*[@id="memo_content"]/div[1]/ul[1]/li[1]/a').click()

    pg.sleep(3)
    try:
        alert=driver.switch_to.alert
        alert.accept()
    except:
        print("발송확인 창 없음")
    
    pg.sleep(3)


    mail_num2+=200
